In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import chess  # https://github.com/niklasf/python-chess
import numpy as np
from chessrl.env import ChessEnv
import torch
board = chess.Board()
# board

In [3]:
# board

In [4]:
env = ChessEnv()
_=env.reset()

In [5]:
from copy import deepcopy

import torch

from chessrl.ppo import PPOAgent, PPOParams

device = torch.device('cuda')
params = PPOParams(in_dim=6*8*8, out_dim=64 * 64)
agent = PPOAgent(params).to(device)
opponent = PPOAgent(params).to(device)
env = ChessEnv(opponent)

def on_episode_learned(episode_number):
    if episode_number % 1 == 0:
        opponent.load_state_dict(deepcopy(agent.state_dict))

In [6]:
agent.learn(env, on_episode_learned)

RuntimeError: CUDA error: CUBLAS_STATUS_INVALID_VALUE when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [8]:
%debug

> /home/spock/.cache/pypoetry/virtualenvs/chessrl-tcytypJB-py3.8/lib/python3.8/site-packages/torch/nn/functional.py(1847)linear()
   1845     if has_torch_function_variadic(input, weight):
   1846         return handle_torch_function(linear, (input, weight), input, weight, bias=bias)
-> 1847     return torch._C._nn.linear(input, weight, bias)
   1848 
   1849 



ipdb>  u


> /home/spock/.cache/pypoetry/virtualenvs/chessrl-tcytypJB-py3.8/lib/python3.8/site-packages/torch/nn/modules/linear.py(96)forward()
     94 
     95     def forward(self, input: Tensor) -> Tensor:
---> 96         return F.linear(input, self.weight, self.bias)
     97 
     98     def extra_repr(self) -> str:



ipdb>  input


 ** On entry to SGEMM  parameter number 10 had an illegal value
tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0., -1., -1., -1., -1., -1., -1., -1., -1.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         -1.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

ipdb>  input.dtype


torch.float32


ipdb>  q


In [ ]:
    device = torch.device('cuda')
    params = Params()

    agent = ChessAgent(params).to(device)
    opponent = ChessAgent(params).to(device)
    env = ChessEnv(params, opponent)

    score = 0.0
    print_interval = 20

    for n_epi in range(10000):
        s = env.reset()
        done = False
        while not done:
            for t in range(params.T_horizon):
                prob = agent.pi(torch.from_numpy(s).float())
                m = Categorical(prob)
                a = m.sample().item()
                s_prime, r, done, info = env.step(a)

                agent.put_data((s, a, r / 100.0, s_prime, prob[a].item(), done))
                s = s_prime

                score += r
                if done:
                    break

            agent.train_net()
        if n_epi % params.n_eps_sync_opponent == 0:
            opponent.load_state_dict(deepcopy(agent.state_dict()))

        if n_epi % print_interval == 0 and n_epi != 0:
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score / print_interval))
            score = 0.0
